# MMVCの学習に必要なconfig系Fileを作成する

ver.2023/1/31

MMVCの学習に必要なファイルを生成します。


​

In [ ]:
#@title ## 0 ノートブックの準備

#@markdown このノートブックのセットアップを行います。セルを実行してください。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown セルの実行が完了したら、次に進んでください。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#debug用ディレクトリの作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
jst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#出力記録用カスタムマジック %%ccapture
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from IPython.utils.capture import capture_output

@magic_arguments.magic_arguments()
@magic_arguments.argument('output', type=str, default='', nargs='?')

@register_cell_magic
def ccapture(line, cell):
    args = magic_arguments.parse_argstring(ccapture, line)
    with capture_output() as outputs:
        get_ipython().run_cell(cell)
    if args.output:
        get_ipython().user_ns[args.output] = outputs
    
    outputs()

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
#git-lfs 事前学習済みモデルのダウンロードに使用
!apt install git-lfs
#os ディレクトリの存在確認に使用
import os
#sys Pythonの管理全般(クラッシュ、tracebackなど)
import sys
print("log: パッケージのインストールが完了しました。\n")


​

In [ ]:
%%ccapture one_prepare_notebook
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
#pyopenjtalk configファイルの作成に使用
print("info: 次のパッケージのインストールには時間がかかります。気長にお待ちください。")
!pip install pydub
!pip install pyworld
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
%%ccapture two_prepare_repo
#@title ## 2 リポジトリの準備
#@markdown リポジトリの準備を行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、Google Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、ローカルのパスを指定してください。


#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

#---関数終わり---

def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  global path_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  path_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer-v1.5.0.0_SiFiGAN', placeholder='/content/drive/MyDrive/MMVC_Trainer-v1.5.0.0_SiFiGAN', description='Path:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, path_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global platform #グローバル指定
  platform = platform_input.value
  global path #グローバル指定
  path = path_input.value
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    %cd $path
  elif platform == "Local":
    %cd $path

#実行
first_view()

​

In [ ]:
%%ccapture three_create_config
#@title ## 3 config系Fileを作成する
#@markdown 学習に必要なファイルを実行します。

#@markdown まず、セルを実行してUIを表示させてください。

#@markdown 次に、以下のオプションを必要に応じて変更して、次へをクリックしてください。

#@markdown 実行時に  
#@markdown　**「WARNING: JPCommonLabel_insert_pause() in jpcommon_label.c: First mora should not be short pause.」**  
#@markdown と表示されますが、無視して大丈夫です。 


#@markdown ​
#@markdown ### Settings
#@markdown config_name：作成するconfig系Fileの名前です。  よく分からない場合は変更不要です。

#@markdown sample_rate：サンプリングレートです。datasetの音声のサンプリングレートに合わせて指定ください。よく分からない場合は変更不要です。  

#@markdown character_select：ターゲットとなるキャラクターの設定です。
#@markdown ずんだもんの学習を行いたい場合は「101」 、
#@markdown 九州そらの学習を行いたい場合は「102」、
#@markdown 四国めたんの学習を行いたい場合は「103」、  
#@markdown 春日部つむぎの学習を行いたい場合は「104」
#@markdown を設定するとクオリティがあがる**かも**しれません。
#@markdown 非サポートキャラは「108」を指定します。

#@markdown multi_speakers：複数話者の学習を同時に行いたい場合に使用します。
#@markdown multi_speaker_correspondence.txt を ```textful配下のディレクトリ名|0～109の番号``` となるように書き換えた後、オンにしてください。このオプションがtrueの場合、character_selectは無視されます。

#@markdown normalize: 音声変換*を行う場合はチェックを入れ、行わない場合はチェックを外してください。よく分からない場合は変更不要です。

#@markdown normalize_backup：音声変換*時、元の音声ファイルのバックアップを作成する場合はチェックを入れ、作成しない場合はチェックを外してください。よく分からない場合は変更不要です。音声変換を行わない場合、このオプションは無視されます。

#@markdown *サンプリングレートの変換、チャンネル数をモノラルに、開始・終了の無音の削除、音量のノーマライズを行います。

def first_view():
  #widgetsの構成
  global config_name_input #グローバル指定
  global sample_rate_input #グローバル指定
  global character_select_input #グローバル指定
  global multi_speakers_input #グローバル指定
  global normalize_input #グローバル指定
  global normalize_backup_input #グローバル指定
  config_name_input = widgets.Text(value="train_config", placeholder="train_config", description='config_name:', disabled=False)
  sample_rate_input = widgets.IntText(value=24000, placeholder=24000, description='sample_rate:', disabled=False)
  character_select_input = widgets.Dropdown(options=['101', '102', '103', '104', '108'], value='101', description='character_select:', disabled=False)
  multi_speakers_input = widgets.Checkbox(value=False, description='multi_speakers:', disabled=False)
  normalize_input = widgets.Checkbox(value=True, description='normalize:', disabled=False)
  normalize_backup_input = widgets.Checkbox(value=True, description='normalize_backup:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  display(config_name_input, sample_rate_input, character_select_input, multi_speakers_input, normalize_input, normalize_backup_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)


def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global config_name #グローバル指定
  config_name = config_name_input.value
  global sample_rate #グローバル指定
  sample_rate = sample_rate_input.value
  global character_select #グローバル指定
  character_select = character_select_input.value
  global multi_speakers #グローバル指定
  multi_speakers = multi_speakers_input.value
  global normalize #グローバル指定
  normalize = normalize_input.value
  global normalize_backup #グローバル指定
  normalize_backup = normalize_backup_input.value

  #normalizeのTrue/Falseで処理を分ける
  print("configファイルの作成を開始します。")
  %cd {path}
  if normalize == True:
    print("normalize:" + str(normalize))
    !python normalize.py $normalize_backup
  elif normalize == False:
    print("normalize:" + str(normalize))

  #multi_speakersのTrue/Falseで処理を分ける
  if multi_speakers == True:
    print("multi_speakers:" + str(multi_speakers))
    !python create_dataset.py -f $config_name -s $sample_rate -m dataset/multi_speaker_correspondence.txt
  elif multi_speakers == False:
    print("multi_speakers:" + str(multi_speakers))
    !python create_dataset.py -f $config_name -s $sample_rate -t $character_select
  print("configファイルの作成が完了しました。")

#実行
first_view()

​

In [ ]:
%%ccapture four_check
#@title ## 4 確認
#@markdown 以下のようなメッセージが表示されれば、保存が正常に完了しています。
#@markdown ファイルが表示されていない場合、ここまでの処理の出力(実行結果)を確認してみてください。
#@markdown ```
#@markdown Directory: filelists
#@markdown 指定したconfig系Fileの名前_textful.txt
#@markdown 指定したconfig系Fileの名前_textful_val.txt
#@markdown 指定したconfig系Fileの名前_textless.txt
#@markdown 指定したconfig系Fileの名前_val_textless.txt
#@markdown 指定したconfig系Fileの名前_Correspondence.txt
#@markdown
#@markdown Directory: configs
#@markdown baseconfig.json
#@markdown 指定したconfig系Fileの名前.json
#@markdown ```

print("Directory:", path, "/filelists")
!ls -1 {path}/filelists
print("\n")
print("Directory:", path, "configs")
!ls -1 {path}/configs

​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## サポート用ファイルの作成

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル
#@markdown * version.txt (MMVCのバージョンが記載されているファイル)

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
version_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-version_txt.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    try:
      #変数名(str)
      print(ev, end=' : ', file=f) 
      #変数の型(変数名がstrとなっているためevalでkeyに直す)
      print(type(eval(ev)), end=' : ', file=f)
      #変数の内容(変数名がstrとなっているためevalでkeyに直す)
      print(eval(ev), file=f)
    except:
      pass

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {path} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#version.txtの保存
try:
  !cp {path}/version.txt {version_txt}
except:
  pass

#logsとconfigsの保存
#directoryが無かったらmkdirしてno-dic.txt置く
!if [ -d {path}/logs ]; then if [ -z "$(ls {path}/logs)" ]; then touch {path}/logs/no-file.txt;else find . -name "*.log" -exec cp {} ~/mmvc-debug/logs \;; fi;else mkdir /mmvc-debug/logs && touch /mmvc-debug/logs/no-dic.txt; fi
!if [ -d {path}/configs ]; then if [ -z "$(ls {path}/configs)" ]; then touch {path}/configs/no-file.txt;else cp -rp {path}/configs /mmvc-debug/configs; fi;else mkdir /mmvc-debug/configs && touch /mmvc-debug/configs/no-dic.txt; fi

#直近のtracebackの保存
with open('/mmvc-debug/traceback.txt', 'w') as f:
  try:
    print(sys.last_type, sys.last_value, sys.last_traceback, file=f)
  except:
    pass

#ccaptureの保存
!mkdir /mmvc-debug/ccapture
with open('/mmvc-debug/ccapture/one_prepare_notebook.txt', 'w') as f:
  try:
    print(one_prepare_notebook, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/two_prepare_repo.txt', 'w') as f:
  try:
    print(two_prepare_repo, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/three_create_config.txt', 'w') as f:
  try:
    print(three_create_config, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/four_check.txt', 'w') as f:
  try:
    print(four_check, file=f)
  except:
    pass

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)